## Imports

In [2]:
import numpy as np
import scipy as sp
import scipy.io.wavfile

## Settings

In [13]:
fname = 'brilliant.wav'

In [14]:
rate, channels = sp.io.wavfile.read(fname)
channels = channels.copy()
rate, channels.shape

(22050, (14595,))

## Encode

In [15]:
msg = "super secret message!"
msglen = 8 * len(msg)
msglen

168

In [16]:
seglen = int(2 * 2**np.ceil(np.log2(2*msglen)))
segnum = int(np.ceil(channels.shape[0]/seglen))
segnum, seglen

(15, 1024)

In [17]:
if len(channels.shape) == 1:
    channels.resize(segnum*seglen, refcheck=False)
    channels = channels[np.newaxis]
else:
    channels.resize((segnum*seglen, channels.shape[1]), refcheck=False)
    channels = channels.T
channels.shape

(1, 15360)

In [18]:
channels.dtype

dtype('int16')

In [19]:
msgbin = np.ravel([[int(y) for y in format(ord(x), '08b')] for x in msg])
msgPi = msgbin.copy()
msgPi[msgPi == 0] = -1
msgPi = msgPi * -np.pi/2

In [20]:
segs = channels[0].reshape((segnum,seglen))

In [21]:
segs = np.fft.fft(segs)
M = np.abs(segs)
P = np.angle(segs)
print(M[0,:3])
print(P[0,:3])

[ 39859.          18443.44842853  40294.31380713]
[ 0.          2.8657574   2.70047085]


In [22]:
dP = np.diff(P, axis=0)
dP[0,:5]

array([ 0.        , -5.94549832, -5.66979862,  3.56063878,  3.56078846])

In [112]:
segmid = seglen // 2
P[0,-msglen+segmid:segmid] = msgPi
P[0,segmid+1:segmid+1+msglen] = -msgPi[::-1]
for i in range(1, len(P)): P[i] = P[i-1] + dP[i-1]

In [113]:
segs = (M * np.exp(1j * P))

In [114]:
segs = np.fft.ifft(segs).real
channels[0] = segs.ravel().astype(np.int16)

In [115]:
sp.io.wavfile.write('steg_'+fname, rate, channels.T)

## Decode

In [27]:
msglen = 8 * 4
seglen = 2*int(2**np.ceil(np.log2(2*msglen)))
segmid = seglen // 2

In [28]:
if len(channels.shape) == 1:
    x = channels[:seglen]
else:
    x = channels[:seglen,0]
x = (np.angle(np.fft.fft(x))[segmid-msglen:segmid] < 0).astype(np.int8)
x = x.reshape((-1,8)).dot(1 << np.arange(8 - 1, -1, -1))
''.join(np.char.mod('%c',x))

'asdf'